Imports

In [35]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.styles.borders import Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

Function for centering and resizing cells

In [36]:
def centerAndResize(worksheet):
    for column in worksheet.columns:
        maxLength = 0
        columnLetter = column[0].column_letter
        for cell in column:
            cell.alignment = Alignment(horizontal= 'center')
            try:
                if len(str(cell.value)) > maxLength:
                    maxLength = len(cell.value)
            except:
                pass
        adjustedWidth = (maxLength + 2) * 1.2
        worksheet.column_dimensions[columnLetter].width = adjustedWidth

Month Device Worksheet

In [37]:
# Replace file names as needed
PATH_NAME = '/Users/maddiekong/Downloads/'
SESSION_COUNTS_FILE_NAME = PATH_NAME + 'DataAnalyst_Ecom_data_sessionCounts.csv'
ADDS_TO_CART_FILE_NAME = PATH_NAME + 'DataAnalyst_Ecom_data_addsToCart.csv'
OUTPUT_FILE_NAME = PATH_NAME + 'Online Retailer Performance Analysis.xlsx'
# Constants
HEADER_FONT = Font(color = 'FFFFFF')
HEADER_FILL = PatternFill(fgColor = '1D6F42', fill_type = 'solid')
SIDE = Side(style = 'thin')
BORDER = Border(left = SIDE, right = SIDE, top = SIDE, bottom = SIDE)
# Import CSV
sessionCounts = pd.read_csv(SESSION_COUNTS_FILE_NAME)
# Add month & year column 
datetimeIndices = pd.DatetimeIndex(sessionCounts['dim_date'])
sessionCounts['year'] = datetimeIndices.year
sessionCounts['month'] = datetimeIndices.month
# Drop date & browser fields
sessionCounts = sessionCounts.drop(columns = ['dim_browser', 'dim_date'])
# Aggregate on month & device
sessionCountsAggregated = sessionCounts.groupby(['year', 'month', 'dim_deviceCategory'], as_index = False).sum()
# Sort data by date descending
sessionCountsAggregated = sessionCountsAggregated.sort_values(['year', 'month'], ascending = False)
# Create formatted date column
sessionCountsAggregated.insert(2,'Date', sessionCountsAggregated['month'].astype(str) + '/' + sessionCountsAggregated['year'].astype(str))
# Drop year & month column
sessionCountsAggregated.drop(columns= ['year', 'month'], inplace = True)
# Calculate the ECR (transactions/sessions)
sessionCountsAggregated['ECR'] = sessionCountsAggregated['transactions']/sessionCountsAggregated['sessions']
# Write to excel
sessionCountsAggregated.to_excel(OUTPUT_FILE_NAME, index = False)
# Load xlsx in openpyxl for worksheet formatting
workbook = load_workbook(OUTPUT_FILE_NAME)
worksheet = workbook.active
# Update worksheet name
worksheet.title = 'Month Device Stats'
# Update header names
header = ['Date', 'Device Category', 'Sessions', 'Transactions', 'QTY', 'ECR']
colNum = 1
# Cell formatting
for i in header:
    worksheet.cell(row = 1, column = colNum).value = i
    worksheet.cell(row = 1, column = colNum).font = HEADER_FONT
    worksheet.cell(row = 1, column = colNum).fill = HEADER_FILL
    worksheet.cell(row = 1, column = colNum).border = BORDER
    colNum += 1
centerAndResize(worksheet)
# Save
workbook.save(OUTPUT_FILE_NAME)   


Month Over Month Worksheet

In [38]:
# Constants
METRICS = ['addsToCart', 'sessions', 'transactions', 'QTY', 'ECR']
METRIC_DISPLAY_NAMES = ['Adds To Cart', 'Sessions', 'Transactions', 'QTY', 'ECR']
MONTHS = 2 
# Import csv
addToCart = pd.read_csv(ADDS_TO_CART_FILE_NAME)
# Drop device category field
sessionCountsv2 = sessionCounts.drop(columns=['dim_deviceCategory'])
# Group by year & month
sessionCountsv2Aggregated = sessionCountsv2.groupby(['year', 'month'], as_index = False).sum()
# Create ECR field
sessionCountsv2Aggregated['ECR'] = sessionCountsv2Aggregated['transactions']/sessionCountsv2Aggregated['sessions']
# Joining metrics from sessionCount sheet
monthOverMonth = pd.merge(addToCart,sessionCountsv2Aggregated, left_on=['dim_year','dim_month'], right_on=['year','month'])
dim = ['year', 'month']
# Drop duplicate year & month
monthOverMonthMetricsOnly = monthOverMonth.drop(columns= dim)
# Select most recent 2 months
monthOverMonthMetricsOnly = monthOverMonthMetricsOnly.sort_values(by=['dim_year','dim_month'], ascending = False).head(MONTHS)
# Load workbook
workbook = load_workbook(OUTPUT_FILE_NAME)
# Create new worksheet
worksheet = workbook.create_sheet('Month Over Month')
# Writing headers
worksheet.cell(row = 1, column = 2).value = '{0}/{1}'.format(monthOverMonthMetricsOnly.iloc[0]['dim_month'].astype(int), monthOverMonthMetricsOnly.iloc[0]['dim_year'].astype(int))
worksheet.cell(row = 1, column = 3).value = '{0}/{1}'.format(monthOverMonthMetricsOnly.iloc[1]['dim_month'].astype(int), monthOverMonthMetricsOnly.iloc[1]['dim_year'].astype(int))
worksheet.cell(row = 1, column = 4).value = 'Absolute Diff'
worksheet.cell(row = 1, column = 5).value = 'Relative Diff (%)'
# Header formatting
for i in range(1, 6):
    worksheet.cell(row = 1, column = i).font = HEADER_FONT
    worksheet.cell(row = 1, column = i).fill = HEADER_FILL
    worksheet.cell(row = 1, column = i).border = BORDER
# Writing metric stats & formatting
rowNum = 2
for i in range(len(METRICS)):
    metric = METRICS[i]
    current = monthOverMonthMetricsOnly.iloc[0][metric]
    prior = monthOverMonthMetricsOnly.iloc[1][metric]
    worksheet.cell(row = rowNum, column = 1).value = METRIC_DISPLAY_NAMES[i]
    worksheet.cell(row = rowNum, column = 1).font = HEADER_FONT
    worksheet.cell(row = rowNum, column = 1).fill = HEADER_FILL
    worksheet.cell(row = rowNum, column = 1).border = BORDER
    worksheet.cell(row = rowNum, column = 2).value = current
    worksheet.cell(row = rowNum, column = 3).value = prior
    worksheet.cell(row = rowNum, column = 4).value = current-prior
    worksheet.cell(row = rowNum, column = 5).value = ((current-prior)/prior)*100
    # red font if number is negative
    if worksheet.cell(row = rowNum, column = 5).value < 0:
        worksheet.cell(row = rowNum, column = 4).font = Font(color = 'ff0000', bold = True)
        worksheet.cell(row = rowNum, column = 5).font = Font(color = 'ff0000', bold = True)
    rowNum += 1
centerAndResize(worksheet)
# Save
workbook.save(OUTPUT_FILE_NAME)    